In [1]:
import requests
import json
import romkan
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm

In [72]:

# 设置 Chrome 浏览器选项
# options = Options()
# options.add_argument("--headless")  # 无头模式（不打开浏览器窗口）
# options.add_argument("--disable-blink-features=AutomationControlled")  # 伪装为真实用户
# options.add_argument(
#    "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
# )
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--disable-gpu')
# options.add_argument('--remote-debugging-port=9222')

# 启动浏览器
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=options)

driver = webdriver.Chrome()

search_urls = [
    "https://home.gamer.com.tw/creationDetail.php?sn=4349325",
    "https://home.gamer.com.tw/creationDetail.php?sn=4726074",
    "https://home.gamer.com.tw/creationDetail.php?sn=5944078",
    "https://home.gamer.com.tw/creationDetail.php?sn=5426900",
    "https://home.gamer.com.tw/creationDetail.php?sn=4831500",
    "https://home.gamer.com.tw/creationDetail.php?sn=6016243",
    "https://home.gamer.com.tw/creationDetail.php?sn=5941347",
    "https://home.gamer.com.tw/creationDetail.php?sn=5639854",
    "https://home.gamer.com.tw/creationDetail.php?sn=5545288",
    "https://home.gamer.com.tw/creationDetail.php?sn=6074598",
]

song_data = []

for url in tqdm(search_urls, desc="爬取中..."):
    try:
        # 访问网页
        driver.get(url)

        # 等待特定元素加载完成（例如，等待 class 为 'home_box' 的 div 加载）
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ST1"))
        )

        # 获取 HTML
        html = driver.page_source

        # 解析 HTML
        soup = BeautifulSoup(html, "html.parser")
        song_data.append(soup)

    except Exception as e:
        print(f"访问 {url} 时出错：{e}")

# 关闭浏览器
driver.quit()

爬取中...:   0%|          | 0/10 [00:00<?, ?it/s]

In [74]:
len(song_data)

10

In [76]:
def extract_json(soup):
    lyrics_jp = []
    lyrics_hiragana = []
    lyrics_zh = []
    
    # target_div = soup.find('div', class_='home_box')
    
    # if target_div:
    bold_tags = soup.find_all('b')
    # else:
    #    raise RuntimeError("No <div> with class 'home_box' found.")
    
    filtered_bold_tags = [b for b in bold_tags if not (len(b.contents) == 1 and b.contents[0].name == 'br')]
    filtered_bold_tags2 = []
    for i in range(len(filtered_bold_tags)):
        b_tag = filtered_bold_tags[i]
        # Check if the <b> tag contains a <font> tag
        if '—————' in b_tag.get_text(strip=True):
            break
        if b_tag.find('font'):
            # If it's not the first tag, append its text to the previous <b> tag
            if i > 0:
                previous_b_tag = filtered_bold_tags2[-1]
                previous_b_tag.append(b_tag.get_text())
                filtered_bold_tags2[-1] = previous_b_tag
        else:
            filtered_bold_tags2.append(b_tag)
    
    
    for i, b in enumerate(filtered_bold_tags2):
        text_content = b.get_text(strip=True)
        if i == 0:
            title = text_content
        elif (i-1)%3 == 0:
            lyrics_jp.append(text_content)
        elif (i-1)%3 == 1:
            hiragana_text = romkan.to_hiragana(text_content)
            lyrics_hiragana.append(hiragana_text)
        else:
            lyrics_zh.append(text_content)    
    # 提取作词、作曲、编曲、演唱者
    info = {}
    for div in soup.find_all("div"):
        text = div.get_text(strip=True)
        if "作詞：" in text:
            info["lyricist"] = text.replace("作詞：", "").strip()
        if "作曲：" in text:
            info["composer"] = text.replace("作曲：", "").strip()
        if "編曲：" in text:
            info["arranger"] = text.replace("編曲：", "").strip()
        if "歌：" in text:
            info["singer"] = text.replace("歌：", "").strip()
    
    # 组织数据结构
    song_json_data = {
        "song": {
            "title": title,
            "lyrics_info": info,
            "lyrics": {
                "japanese": lyrics_jp,
                "hiragana": lyrics_hiragana,
                "chinese": lyrics_zh
            }
        }
    }

    return song_json_data

In [3]:
import re
from collections import defaultdict

def turn_txt_to_json(file, order, title, hiragana=False, info=None):
    lyrics = defaultdict(list)
    jp, zh, hira = order
    base = 4 if hiragana else 3
    with open(file, "r", encoding="utf-8") as file:
        i = 0
        for line in file:
            remove_chars = "!@#$%^&*()-+"
            clean_text = line.translate(str.maketrans("", "", remove_chars)).replace("\n", "").replace("\t", "").replace("\u3000", "")
            clean_text = re.sub(r"\s+", " ", clean_text).strip()
            ind = (i % base)
            if len(clean_text) > 0:
                if (ind in order):
                    if base == 3 and (ind == hira):
                        clean_text = romkan.to_hiragana(clean_text)
                    lyrics[ind].append(clean_text)
                i += 1
                
    lyrics_jp = lyrics[jp]
    lyrics_zh = lyrics[zh]
    lyrics_hira = lyrics[hira]
    song_json_data = {
        "song": {
            "title": title,
            "lyrics_info": info,
            "lyrics": {
                "japanese": lyrics_jp,
                "hiragana": lyrics_hira,
                "chinese": lyrics_zh
            }
        }
    }
    return song_json_data

In [17]:
def turn_multiple_txt_to_json(file_jp, file_zh, file_hira, title, file_roman=None, info=None):
    lyrics_jp = []
    remove_chars = "@#$%^&*-+"
    with open(file_jp, "r", encoding="utf-8") as file:
        for line in file:
            clean_text = line.translate(str.maketrans("", "", remove_chars)).replace("\n", "").replace("\t", "").replace("\u3000", "")
            clean_text = re.sub(r"\s+", " ", clean_text).strip()
            lyrics_jp.append(clean_text)

    lyrics_zh = []
    with open(file_zh, "r", encoding="utf-8") as file:
        for line in file:
            clean_text = line.translate(str.maketrans("", "", remove_chars)).replace("\n", "").replace("\t", "").replace("\u3000", "")
            clean_text = re.sub(r"\s+", " ", clean_text).strip()
            lyrics_zh.append(clean_text)

    lyrics_hira = []
    if file_hira is not None:
        with open(file_hira, "r", encoding="utf-8") as file:
            for line in file:
                clean_text = line.translate(str.maketrans("", "", remove_chars)).replace("\n", "").replace("\t", "").replace("\u3000", "")
                clean_text = re.sub(r"\s+", " ", clean_text).strip()
                lyrics_hira.append(clean_text)
    else:
        with open(file_roman, "r", encoding="utf-8") as file:
            for line in file:
                clean_text = line.translate(str.maketrans("", "", remove_chars)).replace("\n", "").replace("\t", "").replace("\u3000", "")
                clean_text = re.sub(r"\s+", " ", clean_text).strip()
                clean_text = romkan.to_hiragana(clean_text)
                lyrics_hira.append(clean_text)
        
    song_json_data = {
        "song": {
            "title": title,
            "lyrics_info": info,
            "lyrics": {
                "japanese": lyrics_jp,
                "hiragana": lyrics_hira,
                "chinese": lyrics_zh
            }
        }
    }
    return song_json_data

In [21]:
file_jp = "dandadan/jp.txt"
file_zh = "dandadan/zh.txt"
file_hira = "dandadan/hira.txt"
title = "オトネケ"
song_json = turn_multiple_txt_to_json(file_jp, file_zh, file_hira, title, info=None)

In [23]:
song_json

{'song': {'title': 'オトネケ',
  'lyrics_info': None,
  'lyrics': {'japanese': ['諦めの悪い輩',
    'アンタらなんかじゃ束なっても敵わん',
    'くわばらくわばらくわばら目にも止まらん速さ',
    'くたばらん黙らん下がらん押し通す我儘',
    'そこどきな邪魔だ俺はもう1人の貴方',
    '貞ちゃん伽椰ちゃんわんさか黄泉の国wonderland',
    '御祈祷中に何だが4時44分まわったら',
    '四尺四寸四分様がカミナッチャbang around',
    '呼ぶ声がしたんなら文字通りお憑かれさまやん…',
    'ハイレタハイレタハイレタハイレタハイレタ',
    '必死で這い出た先で霧は晴れた',
    'デコとボコが上手く噛み合ったら',
    '痛みが重なったら',
    'ココロカラダアタマ',
    'みなぎってゆく何だか',
    '背中に今羽が生えたならば',
    '暗闇からおさらば',
    '飛び立っていく彼方',
    'ココロカラダアタマ',
    '懐かしい暖かさ',
    '足元に今花が咲いたならば',
    '暗闇からおさらば',
    '飛び立っていく彼方',
    '何度だって生きる',
    'お前や君の中',
    '瞼の裏や耳の中',
    '胸の奥に居着いてるメロディー、リズムに',
    'ダンダダンダンダダンダンダダンダンダダン…',
    '今日も賽の河原ど真ん中',
    '積み上げてくtop of top',
    '鬼とチャンバラ',
    '渡る大海原',
    '鼻歌singin’ shalala',
    '祓いたいのなら末代までの札束(okay？)',
    '誰が開いたか禁(パン)后(ドラ)後は何があっても知らんがな',
    '何百年待ったか超久しぶりの娑婆だ',
    'ガキや若葉',
    'まだコッチ来んじゃねーよバカが',
    '今確かに目が合ったな',
    'こーゆーことかよ…シャマラン…',
    'ハイレタハイレタハイレタハイレタハイレタ',
    '眠り飽きた先で君が待ってた',
    '盾と矛が肩

In [35]:
file = "bomb.txt"
order = (0,2,1)
title = "爆弾魔"
song_json = turn_txt_to_json(file, order, title, hiragana=False, info=None)

In [37]:
song_json

{'song': {'title': '爆弾魔',
  'lyrics_info': None,
  'lyrics': {'japanese': ['死んだ眼で爆弾片手に口を開く',
    'さよならだ人類、みんな吹き飛んじまえ',
    '泣いた顔で爆弾片手 夜が苦しい',
    '安っぽいナイトショーのワンシーンみたいな夢が見たい',
    '今日も出来ませんでした',
    '今日もやれませんでした',
    '青春の全部を爆破したい',
    '君のことを歌にしたい',
    'この日々を爆破して',
    '心ごと爆破して',
    'ずるいよ、優しさってやつちらつかせてさ ずるいよ全部',
    'この部屋を爆破したい',
    '夢がなきゃ生きられない',
    'だから今、さよならだ',
    '吹き飛んじまえ',
    '死んだ目で爆弾片手に街を歩く',
    '誰も見向きもしないんだ 爆弾を翳したとて',
    'ずっと泣けませんでした',
    'ずっと笑えませんでした',
    '青春の全部に君がいる',
    '風が吹けば花が咲く',
    'あの夏を爆破して',
    '思い出を爆破して',
    '酷いよ、君自身は黙って消えたくせに',
    '酷いよ全部',
    'この街を爆破したい',
    'このままじゃ生きられない',
    'だから今、さよならだ',
    '吹き飛んじまえ',
    'もっと笑えばよかった',
    'ずっと戻りたかった',
    '青春の全部に散れば咲け',
    '散れば咲けよ百日紅',
    'この日々を爆破して',
    '心ごと爆破して',
    '辛くてもいい 苦しさも全部僕のものだ わかってるんだ',
    'この星を爆破したい',
    '君を消せるだけいい',
    '今しかない、いなくなれ',
    'この日々を爆破して',
    '心ごと爆破して',
    'ずるいよ、優しさってやつちらつかせてさ ずるいよ全部',
    'この夜を爆破したい',
    '君だけを覚えていたい',
    'だから今、さよならだ',
    '吹き飛んじまえ'],
   'hiragana': ['しんだ め で ば

In [39]:
import json

file_path = "lyrics_with_hiragana.json"

def merge_json(json_list, file_name):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            old_dict = json.load(file)  # 读取已有 JSON 数据
    except FileNotFoundError:
        print(f"⚠️ File '{file_path}' not found. Creating a new dictionary.")
        old_dict = {"song": []}  # Initialize with an empty structure
    except json.JSONDecodeError:
        print(f"⚠️ File '{file_path}' is corrupted or empty. Resetting JSON data.")
        old_dict = {"song": []}  # Reset to a valid empty structure


    for json_data in json_list:
        song = json_data["song"]
        old_dict["song"].append(song)

    # 保存为 JSON 文件
    with open(file_name, "w", encoding="utf-8") as json_file:
        json.dump(old_dict, json_file, ensure_ascii=False, indent=4)
    
    print("歌词数据已成功提取，并存储为 JSON 文件！")

In [41]:
merge_json([song_json], file_path)

歌词数据已成功提取，并存储为 JSON 文件！
